# 9.2. Алгоритм KMeans

На лекции вам была предложена реализация k-means "руками", без применения библиотечных реализаций. Но эта реализация была предложена исключительно для двумерных векторов. Ваша задача - написать класс `KMeans`, в рамках которого должны быть реализованы следующие функции:

* `.fit(X, n_clusters)` - функция вычисления оптимальных центров кластеров, соответствующих логике алгоритма KMeans
* `.predict(X)` - функция кластеризации объектов из выборки `X`

Конструктор класса `KMeans` должен принимать 2 аргумента:
* `K` - количество кластеров
* `init` - аргумент, принимающий массив размерности $K × M$, где $M$ - размерность векторов признаковых описаний объектов, а $K$ - число кластеров, содержащий координаты исходным приближений центров кластеров

Алгоритм не должен содержать элементов готовых решений из любых библиотек, пользоваться можно только библиотеками `numpy` и `collections`. В качестве метрики используйте обычное [евклидово расстояние](https://colab.research.google.com/drive/1BODfwl4F3c7h0CE-t88zavkZWyNe8n5G#scrollTo=9M9zA6jeFDfb).

В качестве критерия останова алгоритма можно использовать следующее утверждение: *алгоритм кластеризации сходится (останавливается), когда изменение центров кластеров на очередной итерации алгоритма незначительно. Т.е. **все попарные расстояния между обновлёнными и предшествующими значениями центров кластеров на очередной итерации не превосходят некоторого маленького числа (в нашем случае рассмотрим 0.001)**.*

В математических терминах это можно записать так. Пусть $\vec{C^i_1}, \vec{C^i_2}, ... \vec{C^i_K}$ - центры кластеров, полученные на итерации алгоритма с номером $i$, а $\vec{C^{i-1}_1}, \vec{C^{i-1}_2}, ... \vec{C^{i-1}_K}$ - центры кластеров, полученные на предыдущей итерации с номером $i-1$. Тогда алгоритм кластеризации завершится после итерации i, если:

$$max\left(||\vec{C^i_1}-\vec{C^{i-1}_1}||, ||\vec{C^i_2}-\vec{C^{i-1}_2}||, ... ||\vec{C^i_K}-\vec{C^{i-1}_K}||\right)\leq0.001$$

In [ ]:
import numpy as np
from collections import defaultdict

class KMeans(object):
    def __init__(self, K, init):
        self.K = K
        self.centroids = np.array(init)

    def fit(self, X):
    # continue until convergence
        while True:
            # dictionary to store cluster points
            clusters = defaultdict(list)
            
            # assign points to clusters
            for point in X:
                cluster = np.argmin([np.linalg.norm(point - c) for c in self.centroids])
                clusters[cluster].append(point)
            
            # calculate new centroids
            new_centroids = np.zeros_like(self.centroids)
            for i, cluster in clusters.items():
                new_centroids[i] = np.mean(cluster, axis=0)
            
            # check for convergence
            diff = np.linalg.norm(new_centroids - self.centroids)
            if diff <= 0.001:
                break
            
            # update centroids
            self.centroids = new_centroids
        
    def predict(self, X):
        # assign points to clusters
        predictions = []
        for point in X:
            cluster = np.argmin([np.linalg.norm(point - c) for c in self.centroids])
            predictions.append(cluster)
        
        return np.array(predictions)

## Пример входных данных

В данном случае имеется в виду только формат данных. Гарантируется, что в данных будут выделенные кластеры.

In [ ]:
import numpy as np
M = 4
N = 2
X = np.random.randn(N, M)
X

## Примечания

1. Иногда при сдаче решения в системе Яндекс. Контест может возникать ошибка TL. Она обычно возникает либо из-за внутренних задержек тестирующей системы, либо из-за недостаточно быстрого решения. В первом случае можно попробовать отправить решение снова (при этом стоит добавить в начало или конец программы строчку с ключевым словом pass - оно не влияет на логику работы программы, но позволяет повторно загрузить одинаковое решение в систему). Во втором случае нужно попытаться оптимизировать программу - понять, в каких фрагментах программа может работать медленно. Здесь рекомендуем обратить внимание на содержание циклов (в частности, проверьте, что в программе нигде не может возникнуть бесконечных циклов, которые никогда не завершаются).

2. В predict нужно возвращать объект типа numpy.array(), а не классический Python list.

3. Обратите внимание, что в задачах кластеризации в функции fit и predict можно подавать одинаковые наборы данных. Но на стадии fit мы осуществляем обучение, подбирая центры кластеров на основании входных данных. А в predict мы предсказываем метки кластеров уже на основании найденных центров кластеров.